In [1]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

out_path = '../../data/ATIS/raw/test/ASR_voice_samples/'

1.10.0
0.10.0


In [3]:
bundle = torchaudio.pipelines.TACOTRON2_WAVERNN_PHONE_LJSPEECH

processor = bundle.get_text_processor()
tacotron2 = bundle.get_tacotron2().to(device)
vocoder = bundle.get_vocoder().to(device)

In [4]:
data = pd.read_csv('../../data/ATIS/experiments/clean/test/test.tsv',sep='\t')

In [5]:
# rearranging rows of dataframe according to sequence length
s = data.TEXT.str.len().sort_values(ascending=False).index
sorted_data = data.reindex(s)

In [6]:
sorted_data =  sorted_data.reset_index(drop=True)
sorted_data.head()

,ID,TEXT,INTENT,INTENT_ID
0,127,show me flight us 1207 from indianapolis to ch...,atis_flight,9
1,383,find a flight between st. petersburg and charl...,atis_flight,9
2,874,please find a flight round trip from los angel...,atis_flight,9
3,316,show me airlines that have flights between tor...,atis_airline,3
4,73,on tuesday are the flights from san diego cali...,atis_flight,9


In [7]:
sorted_sequence = list(sorted_data['TEXT'])

In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [9]:
chunked_sequence = chunks(sorted_sequence,32)

In [ ]:
for idx,chunk in enumerate(list(chunked_sequence)):
    
    print('processing {} chunk'.format(idx))
    
    with torch.inference_mode():
        processed, lengths = processor(chunk)
        processed = processed.to(device)
        lengths = lengths.to(device)
        spec, spec_lengths, _ = tacotron2.infer(processed, lengths)
        waveforms, lengths = vocoder(spec, spec_lengths)
    
    for idx in range(len(chunk)):
        torchaudio.save(out_path + "/{}.wav".format(idx), waveforms[idx:idx+1].cpu(), sample_rate=vocoder.sample_rate)

processing 0 chunk
